# 1. Word2Vec 모델 학습을 위한 전처리


## 1) Tag 데이터 로드

먼저 태그데이터를 읽어옵니다. <br>
pandas를 사용하여 읽어온 데이터를 데이터프레임 형태로 변환합니다. <br>
데이터는 MovieLens의 데이터셋을 사용하였습니다. <br>


In [1]:
import pandas as pd

tag = pd.read_csv('./data/tags_large.csv')
tag_df = pd.DataFrame(tag)
tag_df.head(10)

# 유저별 영화별 태그 데이터입니다.


,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195
5,14,318,justice,1442615192
6,14,480,Dinosaurs,1443148563
7,14,593,psychothriller,1444014286
8,14,1682,philosophy,1442615158
9,14,1682,surveillance,1442615167


## 2) 유저별 태그데이터 배열 생성
로드한 데이터를 잘라서 유저별 영화별 배열을 생성합니다. <br>

<br>

#### 예 : <br>
#### [유저1-영화1-태그1  ,  유저1-영화1-태그2  ,  유저1-영화1-태그3 , ...] <br>
가장 먼저 태그들이 정렬됩니다. 이때, 태그는 시간의 순서에 따라 정렬합니다. <br>
비슷한시간대에 비슷한 태그를 작성한다는 가정하에 정렬하였습니다. <br>

<br>

#### [유저1-영화2-태그1  ,  유저1-영화2-태그2  ,  유저1-영화2-태그3 , ...] <br>
영화가 바뀌면 새로운 어레이가 생성됩니다. <br>
유저1이 영화1번, 영화2번에 작성한 태그는 각각 다른 어레이에 들어갑니다. <br>

<br>

#### [유저2-영화1-태그1  ,  유저2-영화1-태그2  ,  유저2-영화1-태그3 , ...] <br>
유저 1이 작성한 태그가 끝나면 유저2의 태그가 시작됩니다. <br>
마찬가지로 같은영화에 작성한 태그는 같은 어레이에 들어가고 <br>
다른영화에 작성한 태그는 다른 어레이에 들어갑니다. <br>
유저2가 끝나면 유저n 까지 반복합니다. <br> 


In [2]:
from src.word2vec_mylib import tag_manager as tm

tag_array = tm.get_tag_by_user(tag_df, start_movie=110, start_user=14)

for i in range(0, 10):
    print(tag_array[i])
    
# 위의 방식대로 처리한 태그데이터가 담긴 배열입니다.

['medieval', 'epic']
['sci-fi', 'space action']
['justice', 'imdb top 250']
['dinosaurs']
['psychothriller']
['philosophy', 'surveillance']
['epic']
['pixar']
['dinosaurs']
['sci-fi', 'classic sci-fi', 'harrison ford', 'must see']


# 2. Word2Vec 모델 학습


## 1) 모델 학습
<br>
위처럼 생성한 배열을 Word2Vec 모델에 넣어 학습시킵니다. <br>
Word2Vec 모델은 Gensim의 라이브러리를 사용하였습니다. <br> 
<br>
Skip gram 방식을 사용하였고, Window Size는 256 차원입니다. <br>
100번 이하로 나온 태그들은 제거하고, 100번 학습을 시켰습니다. <br>


In [3]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec(tag_array,
                 size=256,
                 window=3,
                 workers=8,
                 min_count=100,
                 iter=100,
                 sg=1)

model.init_sims(replace=True)


C:\Users\ASUS\Desktop\Github\Movie-Recommender\venv\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## 2) 모델 테스트

<br>

모델 학습이 잘 되었는지 테스트를 해봅니다. <br>
marvel 태그와 비슷한 태그를 출력합니다. <br>

In [5]:
print("marvel 태그와 유사한 태그 목록 :  \n")
model.wv.most_similar("marvel")


marvel 태그와 유사한 태그 목록 :  



C:\Users\ASUS\Desktop\Github\Movie-Recommender\venv\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('marvel cinematic universe', 0.568035900592804),
 ('mcu', 0.5632034540176392),
 ('superhero', 0.5316859483718872),
 ('the avengers', 0.5253505706787109),
 ('comic book', 0.5031802654266357),
 ('thor', 0.4992273151874542),
 ('iron man', 0.4754672348499298),
 ('x-men', 0.47319531440734863),
 ('captain america', 0.4463457465171814),
 ('superheroes', 0.4195388853549957)]

# 3. 추천 섹션


In [6]:
from src.word2vec_mylib import movie_manager as mm
from src.word2vec_mylib import recommend_manager as rm
from src.word2vec_mylib import tag_manager as tm

tag = pd.read_csv('./data/tags_large.csv')
tag_df = pd.DataFrame(tag)
tag_df = mm.cut_userId_timestamp(tag_df)

rating = pd.read_csv('./data/ratings_large.csv')
rating_df = pd.DataFrame(rating)
rating_df = mm.get_higher_rating_movie(rating_df)

movie = pd.read_csv('./data/movies_large.csv')
movie_df = pd.DataFrame(movie)

In [ ]:
testMovieId = 586
# 테스트 영화 번호

testMovie_all_tags = tm.get_all_tags_by_freq(testMovieId, tag_df)
# 테스트 영화의 모든 태그를 빈도순으로 뽑아냄
testMovieTF_tags = tm.get_TF_tags(testMovie_all_tags)
# 가중치가 10퍼센트 이상인 TF 태그 목록을 생성함 <-- 테스트 영화의 대표태그들을 의미함
testMovie_model_tags = tm.get_recommend_tags(testMovie_all_tags, model)
# 가중치가 10처센트 이상 + w2v모델 유사태그을 합친 태그 목록을 생성함 <-- 추천해줄만한 영화를 걸러내기 위해 사용됨
recommend_movies_array = mm.get_recommend_movie_list(tag_df, rating_df, testMovie_model_tags)
# 추천영화목록을 만듬 <-- 평균평점 3점이상이고, 테스트영화의 모델 태그를 하나라도 보유한 영화를 뜻함.
recommend_movies_with_tags = mm.get_recommend_movie_with_all_tags(tag_df, recommend_movies_array)
# 추천영화목록의 영화들과 모든 태그를 보유한 데이터프레임을 생성함
recommend_TF = tm.get_TF_tags_for_many(recommend_movies_with_tags)
# 추천영화들의 가중치 10퍼센트 이상의 태그 목록을 생성함
result_df = rm.recommend(recommend_TF, testMovieTF_tags, model)
# 추천영화들의 태그와 테스트영화의 태그를 n x n 으로 매칭해서
# 유사도를 구해 그 순서로 정렬한 데이터 프레임을 뽑아냄

result_df = pd.merge(result_df, movie_df)
result_df = pd.merge(result_df, rating_df)
# 모든 프레임을 합침

print(mm.id_to_name(testMovieId, movie_df), "\n")
print(testMovieTF_tags, "\n")
result_df = result_df[['movieId', 'similarity', 'rating', 'title', 'genres', 'tags']]
result_df = result_df[result_df.movieId != testMovieId] # 리스트에 현재 영화가 있었다면, 잘라냄
result_df

C:\Users\ASUS\Desktop\Github\Movie-Recommender\venv\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


     movieId              title           genres
580      586  Home Alone (1990)  Children|Comedy 

['christmas', 'childhood classics', 'nostalgia', 'family', 'macaulay culkin', 'for kids'] 



,movieId,similarity,rating,title,genres,tags
0,136588,0.685600,3.512987,Santa Claus is Comin' to Town (1970),Animation|Children|Fantasy,"[christmas, santa claus, stop motion, claymati..."
2,128679,0.565736,3.655556,The Year Without a Santa Claus (1974),Animation|Children,"[christmas, santa claus, sibling relationship,..."
3,60030,0.547984,3.237500,Spiral (2007),Drama|Horror|Thriller,"[waitress, party, painter, oregon, notebook, n..."
4,152641,0.499738,4.000000,Pinocchio's Christmas (1980),Animation|Children,"[made for tv, christmas, stop-motion, rankin/b..."
5,105540,0.481719,3.318182,"All Dogs Christmas Carol, An (1998)",Animation|Children|Comedy|Musical,"[christmas, musical, author:charles dickens, a..."
6,169948,0.476902,3.688525,Tomorrow Everything Starts (2016),Comedy|Drama,"[french, omar sy, absent parent, remake, twist..."
7,168042,0.476287,3.550000,A December Bride (2016),Romance,"[christmas, holiday, romantic comedy, comedy]"
8,128725,0.471427,3.600000,Eloise at Christmastime (2003),Children|Comedy|Drama,"[christmas, holiday, waiter, wedding, hotel, l..."
9,178827,0.467361,3.709350,Paddington 2 (2017),Adventure|Animation|Children|Comedy,"[family, sequel, talking animals, reviewed, lo..."
10,96283,0.465566,3.168421,Diary of a Wimpy Kid: Dog Days (2012),Children|Comedy,"[summer, family, fun, kids like, based on chil..."


C:\Users\ASUS\Desktop\Github\Movie-Recommender\venv\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


       movieId                           title                   genres
27550   122892  Avengers: Age of Ultron (2015)  Action|Adventure|Sci-Fi 

['superhero', 'marvel'] 



,movieId,similarity,rating,title,genres,tags
0,8636,1.634517,3.453454,Spider-Man 2 (2004),Action|Adventure|Sci-Fi|IMAX,"[superhero, comic book, marvel, super-hero, ac..."
1,165865,1.260785,3.600000,Marvel Studios: Assembling a Universe (2014),Documentary,"[marvel cinematic universe, superhero]"
2,120833,1.073111,3.333333,Super Capers (2009),Action|Adventure|Comedy|Fantasy|Sci-Fi,[superhero]
3,3905,0.899950,3.231293,"Specials, The (2000)",Comedy,"[less than 300 ratings, superhero, mockumentar..."
4,58559,0.880916,4.173756,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,"[heath ledger, batman, superhero, christian ba..."
5,155078,0.814738,3.611111,Lo chiamavano Jeeg Robot (2016),Fantasy,"[action, super strength, funny, rome, antihero..."
6,187213,0.811582,3.272727,Psychokinesis (2018),Action|Adventure|Fantasy,"[superhero, ridiculous, netflix, funny]"
7,76210,0.681917,3.343750,Defendor (2009),Comedy|Crime|Drama,"[superhero, woody harrelson, kat dennings, vig..."
8,107559,0.646178,3.410714,Am Ende eiens viel zu kurzen Tages (Death of a...,Animation|Drama,"[nudity, sexually graphic, superhero]"
